In [2]:
import pandas as pd
from pathlib import Path  

In [55]:
import argparse
import json
import os
import pandas as pd
import numpy as np
import yaml
from sklearn.metrics import log_loss, accuracy_score

from src.model import make_dataset, build_pipeline, evaluate_cv, predict_with_ev, save_pipeline

In [67]:
df = pd.read_csv("data/matches.csv", sep=';', on_bad_lines='skip')

In [68]:
df.head()

,Game_Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,16/08/2024,20:00,Man United,Fulham,1,0,H,0,0,D,...,1.86,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08
1,17/08/2024,12:30,Ipswich,Liverpool,0,2,A,0,0,D,...,2.05,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93
2,17/08/2024,15:00,Arsenal,Wolves,2,0,H,1,0,H,...,2.02,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96
3,17/08/2024,15:00,Everton,Brighton,0,3,A,0,1,A,...,1.87,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11
4,17/08/2024,15:00,Newcastle,Southampton,1,0,H,1,0,H,...,1.87,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10


In [69]:
df.shape

(380, 119)

In [70]:
print(df)

      Game_Date   Time       HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  \
0    16/08/2024  20:00     Man United       Fulham     1     0   H     0   
1    17/08/2024  12:30        Ipswich    Liverpool     0     2   A     0   
2    17/08/2024  15:00        Arsenal       Wolves     2     0   H     1   
3    17/08/2024  15:00        Everton     Brighton     0     3   A     0   
4    17/08/2024  15:00      Newcastle  Southampton     1     0   H     1   
..          ...    ...            ...          ...   ...   ...  ..   ...   
375  25/05/2025  16:00      Newcastle      Everton     0     1   A     0   
376  25/05/2025  16:00  Nott'm Forest      Chelsea     0     1   A     0   
377  25/05/2025  16:00    Southampton      Arsenal     1     2   A     0   
378  25/05/2025  16:00      Tottenham     Brighton     1     4   A     1   
379  25/05/2025  16:00         Wolves    Brentford     1     1   D     0   

     HTAG HTR  ... B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  MaxCAHA  \
0       0   D 

In [61]:
print(df.columns.tolist())

['Game_Date;Time;HomeTeam;AwayTeam;FTHG;FTAG;FTR;HTHG;HTAG;HTR;Referee;HS;AS;HST;AST;HF;AF;HC;AC;HY;AY;HR;AR;B365H;B365D;B365A;BWH;BWD;BWA;BFH;BFD;BFA;PSH;PSD;PSA;WHH;WHD;WHA;1XBH;1XBD;1XBA;MaxH;MaxD;MaxA;AvgH;AvgD;AvgA;BFEH;BFED;BFEA;B365>2.5;B365<2.5;P>2.5;P<2.5;Max>2.5;Max<2.5;Avg>2.5;Avg<2.5;BFE>2.5;BFE<2.5;AHh;B365AHH;B365AHA;PAHH;PAHA;MaxAHH;MaxAHA;AvgAHH;AvgAHA;BFEAHH;BFEAHA;B365CH;B365CD;B365CA;BWCH;BWCD;BWCA;BFCH;BFCD;BFCA;PSCH;PSCD;PSCA;WHCH;WHCD;WHCA;1XBCH;1XBCD;1XBCA;MaxCH;MaxCD;MaxCA;AvgCH;AvgCD;AvgCA;BFECH;BFECD;BFECA;B365C>2.5;B365C<2.5;PC>2.5;PC<2.5;MaxC>2.5;MaxC<2.5;AvgC>2.5;AvgC<2.5;BFEC>2.5;BFEC<2.5;AHCh;B365CAHH;B365CAHA;PCAHH;PCAHA;MaxCAHH;MaxCAHA;AvgCAHH;AvgCAHA;BFECAHH;BFECAHA']


In [36]:
df.columns = df.columns.str.strip()

In [71]:
print(df.columns)

Index(['Game_Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR',
       ...
       'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA',
       'AvgCAHH', 'AvgCAHA', 'BFECAHH', 'BFECAHA'],
      dtype='object', length=119)


In [72]:
print(df.columns.tolist())

['Game_Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'BFH', 'BFD', 'BFA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', '1XBH', '1XBD', '1XBA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'BFEH', 'BFED', 'BFEA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'BFE>2.5', 'BFE<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'BFEAHH', 'BFEAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'BFCH', 'BFCD', 'BFCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', '1XBCH', '1XBCD', '1XBCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'BFECH', 'BFECD', 'BFECA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'BFEC>2.5', 'BFEC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', '

In [73]:
df.columns

Index(['Game_Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR',
       ...
       'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA',
       'AvgCAHH', 'AvgCAHA', 'BFECAHH', 'BFECAHA'],
      dtype='object', length=119)

In [74]:
print(df.shape)

(380, 119)


In [75]:
print(df.head())

    Game_Date   Time    HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  16/08/2024  20:00  Man United       Fulham     1     0   H     0     0   D   
1  17/08/2024  12:30     Ipswich    Liverpool     0     2   A     0     0   D   
2  17/08/2024  15:00     Arsenal       Wolves     2     0   H     1     0   H   
3  17/08/2024  15:00     Everton     Brighton     0     3   A     0     1   A   
4  17/08/2024  15:00   Newcastle  Southampton     1     0   H     1     0   H   

   ... B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  MaxCAHA  AvgCAHH  AvgCAHA  \
0  ...     1.86      2.07   1.83   2.11     1.88     2.11     1.82     2.05   
1  ...     2.05      1.88   2.04   1.90     2.20     2.00     1.99     1.88   
2  ...     2.02      1.91   2.00   1.90     2.05     1.93     1.99     1.87   
3  ...     1.87      2.06   1.86   2.07     1.92     2.10     1.83     2.04   
4  ...     1.87      2.06   1.88   2.06     1.89     2.10     1.82     2.05   

   BFECAHH  BFECAHA  
0     1.90     2

In [76]:
df.isnull().sum()

Game_Date    0
Time         0
HomeTeam     0
AwayTeam     0
FTHG         0
            ..
MaxCAHA      0
AvgCAHH      0
AvgCAHA      0
BFECAHH      0
BFECAHA      0
Length: 119, dtype: int64

In [77]:
print(df.isnull().mean() * 100)

Game_Date    0.0
Time         0.0
HomeTeam     0.0
AwayTeam     0.0
FTHG         0.0
            ... 
MaxCAHA      0.0
AvgCAHH      0.0
AvgCAHA      0.0
BFECAHH      0.0
BFECAHA      0.0
Length: 119, dtype: float64


In [78]:
print("Prediction dataframe columns:", df.columns.tolist())

Prediction dataframe columns: ['Game_Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'BFH', 'BFD', 'BFA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', '1XBH', '1XBD', '1XBA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'BFEH', 'BFED', 'BFEA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'BFE>2.5', 'BFE<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'BFEAHH', 'BFEAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'BFCH', 'BFCD', 'BFCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', '1XBCH', '1XBCD', '1XBCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'BFECH', 'BFECD', 'BFECA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'BFEC>2.5', 'BFEC<2.5', 'AHCh', 'B365CAHH', '

In [79]:
print(df.head(1).T)

                    0
Game_Date  16/08/2024
Time            20:00
HomeTeam   Man United
AwayTeam       Fulham
FTHG                1
...               ...
MaxCAHA          2.11
AvgCAHH          1.82
AvgCAHA          2.05
BFECAHH           1.9
BFECAHA          2.08

[119 rows x 1 columns]
